In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gym
from gym import spaces
from scipy import ndimage

class WoodCuttingEnv(gym.Env):
    def __init__(self, big_platform_size=(100, 100), max_platforms=1000):
        super(WoodCuttingEnv, self).__init__()
        
        # Initialize with the size of big wood platforms
        self.big_platform_width, self.big_platform_height = big_platform_size
        self.max_platforms = max_platforms
        
        # State: Representation of the current platform's state
        # We'll represent it as a binary grid
        self.grid_size = 100  # We'll use a 100x100 grid for simplicity
        # Track piece types in each position (0 = empty, 1+ = piece type index + 1)
        self.platforms = [np.zeros((self.grid_size, self.grid_size), dtype=np.int8)]
        self.piece_types = [np.zeros((self.grid_size, self.grid_size), dtype=np.int8)]


        # State space: Current platform's grid state + remaining order pieces + current platform index
        grid_space = spaces.Box(low=0, high=1, shape=(self.grid_size, self.grid_size), dtype=np.int8)
        
        # Max number of different piece types in an order
        self.max_order_types = 10
        # Order space: (width, height, quantity) for each order type
        order_space = spaces.Box(
            low=np.array([[1, 1, 0]] * self.max_order_types),
            high=np.array([[self.big_platform_width, self.big_platform_height, 100]] * self.max_order_types),
            dtype=np.int32
        )
        
        # Current platform index
        platform_index_space = spaces.Discrete(self.max_platforms + 1)  # +1 for "no platforms left"
        
        # Combine spaces
        self.observation_space = spaces.Dict({
            'grid': grid_space,
            'order': order_space,
            'platform_index': platform_index_space
        })
        
        # Action space: (x, y, piece_type, rotation)
        # x, y: position to place the piece
        # piece_type: which piece type from the order to use
        # rotation: 0 or 1 (0° or 90°)
        self.action_space = spaces.MultiDiscrete([
            self.grid_size,  # x
            self.grid_size,  # y
            self.max_order_types,  # piece_type
            2  # rotation (0 or 1)
        ])
        
        # Initialize state
        self.reset()
    
    def reset(self, order=None):
        """Reset environment with a new or provided order."""
        # Initialize the first platform grid (0 = empty, 1 = filled)
        self.platforms = [np.zeros((self.grid_size, self.grid_size), dtype=np.int8)]
        self.piece_types = [np.zeros((self.grid_size, self.grid_size), dtype=np.int8)]
        self.current_platform_idx = 0
        
        # Generate a random order if none provided
        if order is None:
            self.order = self._generate_random_order()
        else:
            self.order = order.copy()
        
        return self._get_observation()
    
    def _generate_random_order(self):
        """Generate a random cutting order."""
        num_types = np.random.randint(1, self.max_order_types + 1)
        order = []
        
        for _ in range(num_types):
            width = np.random.randint(5, min(self.big_platform_width // 2, 30) + 1)
            height = np.random.randint(5, min(self.big_platform_height // 2, 30) + 1)
            quantity = np.random.randint(1, 20)
            order.append([width, height, quantity])
        
        # Pad the order to max_order_types
        while len(order) < self.max_order_types:
            order.append([0, 0, 0])
            
        return np.array(order)
    
    def _get_observation(self):
        """Return the current observation."""
        return {
            'grid': self.platforms[self.current_platform_idx],
            'order': self.order,
            'platform_index': self.current_platform_idx
        }
    
    def _is_valid_placement(self, x, y, piece_width, piece_height, platform_idx=None):
        """Check if a piece can be placed at (x, y) with given dimensions."""
        if platform_idx is None:
            platform_idx = self.current_platform_idx
            
        if x + piece_width > self.grid_size or y + piece_height > self.grid_size:
            return False
        
        # Check if the area is empty
        if np.any(self.platforms[platform_idx][y:y+piece_height, x:x+piece_width] == 1):
            return False
        
        return True
    
    def _can_fit_anywhere(self, piece_width, piece_height, platform_idx=None):
        """Check if a piece can fit anywhere on the specified platform."""
        if platform_idx is None:
            platform_idx = self.current_platform_idx
            
        for y in range(self.grid_size - piece_height + 1):
            for x in range(self.grid_size - piece_width + 1):
                if self._is_valid_placement(x, y, piece_width, piece_height, platform_idx):
                    return True, (x, y)
        return False, None
    
    def _place_piece(self, x, y, piece_width, piece_height, piece_type, platform_idx=None):
        """Place a piece at (x, y) with given dimensions."""
        if platform_idx is None:
            platform_idx = self.current_platform_idx
            
        self.platforms[platform_idx][y:y+piece_height, x:x+piece_width] = 1
        self.piece_types[platform_idx][y:y+piece_height, x:x+piece_width] = piece_type + 1  # +1 to avoid 0
    
    def calculate_metrics(self):
        """
        Calculate various efficiency metrics for the current cutting solution.
        
        Returns:
            dict: Dictionary containing the following metrics:
                - waste_rate: Total unused area / Total area of all cut stock
                - fitness: Total area of all cut items / Total area of all cut stock
                - platforms_used: Number of platforms used
                - filled_area: Total area filled with cut items
                - total_stock_area: Total area of all cut stock
        """
        # Count how many platforms were actually used
        used_platforms = 0
        filled_area = 0
        
        for platform in self.platforms:
            if np.any(platform == 1):
                used_platforms += 1
                filled_area += np.sum(platform)
        
        # Calculate total stock area (area of all used platforms)
        total_stock_area = self.big_platform_width * self.big_platform_height * used_platforms
        
        # Calculate waste rate
        waste_area = total_stock_area - filled_area
        waste_rate = waste_area / total_stock_area if total_stock_area > 0 else 0
        
        # Calculate fitness
        fitness = filled_area / total_stock_area if total_stock_area > 0 else 0
        
        return {
            'waste_rate': waste_rate,
            'fitness': fitness,
            'platforms_used': used_platforms,
            'filled_area': filled_area,
            'total_stock_area': total_stock_area,
            'waste_area': waste_area
        }
    

    def step(self, action):
        """
        Take a step in the environment.
        
        Args:
            action: [x, y, piece_type, rotation]
        
        Returns:
            observation, reward, done, info
        """
        x, y, piece_type, rotation = action
        
        # Check if piece_type is valid
        if piece_type >= len(self.order) or self.order[piece_type][2] <= 0:
            # Invalid piece type or no more pieces of this type
            return self._get_observation(), -10, False, {'message': 'Invalid piece type'}
        
        # Get piece dimensions
        width, height, quantity = self.order[piece_type]
        
        # Apply rotation if needed
        if rotation == 1:
            width, height = height, width
        
        # Check if placement is valid at the chosen position on the current platform
        if not self._is_valid_placement(x, y, width, height):
            # Try to find placement on any existing platform
            placed = False
            best_position = None
            best_platform = None
            
            # Check all existing platforms
            for platform_idx in range(len(self.platforms)):
                can_fit, new_pos = self._can_fit_anywhere(width, height, platform_idx)
                if can_fit:
                    if best_position is None or platform_idx <= best_platform:
                        best_position = new_pos
                        best_platform = platform_idx
                        placed = True
            
            if placed:
                # Place at the best position found
                x, y = best_position
                self._place_piece(x, y, width, height, piece_type, best_platform)
                self.order[piece_type][2] -= 1
                # Set the current platform to where we placed the piece
                self.current_platform_idx = best_platform
                reward = width * height - 5  # Small penalty for repositioning
            else:
                # Try creating a new platform if allowed
                if self.current_platform_idx + 1 < self.max_platforms:
                    new_platform_idx = len(self.platforms)
                    self.platforms.append(np.zeros((self.grid_size, self.grid_size), dtype=np.int8))
                    self.piece_types.append(np.zeros((self.grid_size, self.grid_size), dtype=np.int8))
                    
                    # Place at the beginning of the new platform
                    if self._is_valid_placement(0, 0, width, height, new_platform_idx):
                        self._place_piece(0, 0, width, height, piece_type, new_platform_idx)
                        self.order[piece_type][2] -= 1
                        self.current_platform_idx = new_platform_idx
                        reward = width * height - 50  # Reward for successful placement minus penalty for new platform
                    else:
                        reward = -20  # Penalty for invalid placement even on new platform
                else:
                    reward = -30  # No more platforms available
                    return self._get_observation(), reward, True, {'message': 'No more platforms'}
        else:
            # Place the piece at the original position on current platform
            self._place_piece(x, y, width, height, piece_type)
            self.order[piece_type][2] -= 1
            reward = width * height  # Reward proportional to the piece area
        
        # Check if all pieces have been placed
        done = np.all(self.order[:, 2] == 0)
        
        if done:
            # Calculate metrics
            metrics = self.calculate_metrics()
            
            # Add final reward based on fitness
            reward += metrics['fitness'] * 1000
            
            return self._get_observation(), reward, done, {
                'message': 'All pieces placed',
                'waste_rate': metrics['waste_rate'],
                'fitness': metrics['fitness'],
                'platforms_used': metrics['platforms_used'],
                'filled_area': metrics['filled_area'],
                'total_stock_area': metrics['total_stock_area']
            }
        
        return self._get_observation(), reward, done, {}
    
    def render(self):
        # """Render the current state of the environment."""
        """Render the current state of the environment with different colors for each piece type."""
        piece_colors = ['white', 'red', 'blue', 'green', 'purple', 'orange', 'yellow', 'black', 'gray', 'pink', 'brown']
        fig, axs = plt.subplots(1, len(self.platforms), figsize=(5*len(self.platforms), 5))

        if len(self.platforms) == 1:
            axs = [axs]
        
        for i, platform in enumerate(self.platforms):
            ax = axs[i]
            
            # Create a colored image
            colored_image = np.zeros((self.grid_size, self.grid_size, 3))
            
            # Fill with colors based on piece types
            for y in range(self.grid_size):
                for x in range(self.grid_size):
                    piece_type = self.piece_types[i][y, x]
                    if piece_type > 0:
                        # Convert color name to RGB
                        color_name = piece_colors[piece_type]
                        color_rgb = np.array(plt.matplotlib.colors.to_rgb(color_name))
                        colored_image[y, x] = color_rgb
            
            ax.imshow(colored_image)
            ax.set_title(f'Platform {i+1}')
            ax.set_xlim(0, self.grid_size)
            ax.set_ylim(0, self.grid_size)
            ax.invert_yaxis()  # Invert y-axis to match grid coordinates
            
            ax.grid(True, color='black', linewidth=0.5, alpha=0.3)
            
            # Add piece outlines
            for type_id in range(1, self.max_order_types + 1):
                piece_mask = (self.piece_types[i] == type_id)
                if not np.any(piece_mask):
                    continue
                    
        # Display remaining order with colors
        order_text = []
        for idx, (w, h, q) in enumerate(self.order):
            if q > 0:
                color = piece_colors[idx + 1]
                order_text.append(f"<span style='color:{color}'>{w}x{h} (qty: {q})</span>")
        
        if order_text:
            from matplotlib.text import Text
            plt.figtext(0.5, 0.01, f"Remaining pieces: {', '.join(order_text)}", 
                    ha="center", fontsize=9, bbox={"facecolor":"white", "alpha":0.8})
        else:
            plt.figtext(0.5, 0.01, "All pieces placed!", ha="center", fontsize=9, 
                    bbox={"facecolor":"green", "alpha":0.5})
        
        plt.tight_layout()
        plt.subplots_adjust(bottom=0.15)
        plt.show()
    def close(self):
        plt.close('all')

In [21]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
import random

class ActorCriticNetwork(nn.Module):
    def __init__(self, input_shape, action_dims):
        super(ActorCriticNetwork, self).__init__()
        
        # CNN for processing the grid
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=2)
        
        # Calculate the size after convolutions
        def conv2d_size_out(size, kernel_size=3, stride=2):
            return (size - (kernel_size - 1) - 1) // stride + 1
        
        conv_width = conv2d_size_out(conv2d_size_out(conv2d_size_out(input_shape[1], 5, 2)))
        conv_height = conv2d_size_out(conv2d_size_out(conv2d_size_out(input_shape[0], 5, 2)))
        linear_input_size = conv_width * conv_height * 64
        
        # FC for processing the order information
        self.fc_order = nn.Linear(30, 128)  # 10 order types x 3 features (width, height, quantity)
        
        # FC for processing platform index
        self.fc_platform = nn.Linear(1, 32)
        
        # Combine and process
        self.fc_combine = nn.Linear(linear_input_size + 128 + 32, 512)
        
        # Actor head (policy)
        self.actor_x = nn.Linear(512, action_dims[0])  # x position
        self.actor_y = nn.Linear(512, action_dims[1])  # y position
        self.actor_piece = nn.Linear(512, action_dims[2])  # piece type
        self.actor_rotation = nn.Linear(512, action_dims[3])  # rotation
        
        # Critic head (value function)
        self.critic = nn.Linear(512, 1)
        
    def forward(self, grid, order, platform_idx):
        # Process grid
        grid = grid.unsqueeze(1)  # Add channel dimension
        grid = F.relu(self.conv1(grid))
        grid = F.relu(self.conv2(grid))
        grid = F.relu(self.conv3(grid))
        grid = grid.view(grid.size(0), -1)  # Flatten
        
        # Process order
        order = order.view(order.size(0), -1)  # Flatten
        order = F.relu(self.fc_order(order))
        
        # Process platform index
        platform_idx = platform_idx.float().view(-1, 1)
        platform = F.relu(self.fc_platform(platform_idx))
        
        # Combine
        combined = torch.cat((grid, order, platform), dim=1)
        features = F.relu(self.fc_combine(combined))
        
        # Actor outputs (policy distributions)
        x_logits = self.actor_x(features)
        y_logits = self.actor_y(features)
        piece_logits = self.actor_piece(features)
        rotation_logits = self.actor_rotation(features)
        
        # Critic output (value function)
        value = self.critic(features)
        
        return x_logits, y_logits, piece_logits, rotation_logits, value

class PPOMemory:
    def __init__(self, batch_size):
        self.states = []
        self.actions = []
        self.probs = []
        self.vals = []
        self.rewards = []
        self.dones = []
        self.batch_size = batch_size
        
    def store_memory(self, state, action, probs, vals, reward, done):
        self.states.append(state)
        self.actions.append(action)
        self.probs.append(probs)
        self.vals.append(vals)
        self.rewards.append(reward)
        self.dones.append(done)
    
    def clear_memory(self):
        self.states = []
        self.actions = []
        self.probs = []
        self.vals = []
        self.rewards = []
        self.dones = []
    
    def generate_batches(self):
        n_states = len(self.states)
        batch_start = np.arange(0, n_states, self.batch_size)
        indices = np.arange(n_states, dtype=np.int64)
        np.random.shuffle(indices)
        batches = [indices[i:i+self.batch_size] for i in batch_start]
        
        return batches

class PPOAgent:
    def __init__(self, state_shape, action_space, device="cuda" if torch.cuda.is_available() else "cpu",
                 lr=0.0003, gamma=0.99, gae_lambda=0.95, policy_clip=0.2, 
                 batch_size=64, n_epochs=10, entropy_coefficient=0.01):
        self.state_shape = state_shape
        self.action_space = action_space
        self.device = device
        
        # PPO hyperparameters
        self.gamma = gamma
        self.policy_clip = policy_clip
        self.n_epochs = n_epochs
        self.gae_lambda = gae_lambda
        self.entropy_coefficient = entropy_coefficient
        
        # Create actor-critic network
        self.actor_critic = ActorCriticNetwork(
            input_shape=(state_shape['grid'][0], state_shape['grid'][1]),
            action_dims=(action_space[0], action_space[1], action_space[2], action_space[3])
        ).to(device)
        
        self.optimizer = optim.Adam(self.actor_critic.parameters(), lr=lr)
        self.memory = PPOMemory(batch_size)
        
        # For handling actions
        self.grid_size = state_shape['grid'][0]
        self.max_order_types = state_shape['order'][0]
        self.rotations = action_space[3]
        
    def choose_action(self, state, training=True):
        # Convert state to tensors
        grid = torch.FloatTensor(state['grid']).unsqueeze(0).to(self.device)
        order = torch.FloatTensor(state['order']).unsqueeze(0).to(self.device)
        platform_idx = torch.LongTensor([state['platform_index']]).to(self.device)
        
        # Get action distributions and value from actor-critic
        with torch.no_grad():
            x_logits, y_logits, piece_logits, rotation_logits, value = self.actor_critic(
                grid, order, platform_idx
            )
        
        # Create distributions for each action component
        x_dist = Categorical(F.softmax(x_logits, dim=1))
        y_dist = Categorical(F.softmax(y_logits, dim=1))
        
        # Mask invalid piece types (pieces with quantity 0)
        piece_mask = torch.ones_like(piece_logits) * float('-inf')
        for i, (_, _, qty) in enumerate(state['order']):
            if qty > 0:
                piece_mask[0, i] = 0  # Unmask valid piece types
        
        masked_piece_logits = piece_logits + piece_mask
        piece_dist = Categorical(F.softmax(masked_piece_logits, dim=1))
        rotation_dist = Categorical(F.softmax(rotation_logits, dim=1))
        
        # Sample actions from distributions (or take most likely action during evaluation)
        if training:
            x = x_dist.sample()
            y = y_dist.sample()
            piece_type = piece_dist.sample()
            rotation = rotation_dist.sample()
        else:
            x = torch.argmax(x_dist.probs)
            y = torch.argmax(y_dist.probs)
            piece_type = torch.argmax(piece_dist.probs)
            rotation = torch.argmax(rotation_dist.probs)
        
        # Calculate log probabilities
        x_prob = x_dist.log_prob(x)
        y_prob = y_dist.log_prob(y)
        piece_prob = piece_dist.log_prob(piece_type)
        rotation_prob = rotation_dist.log_prob(rotation)
        
        # Sum the log probs to get the total action log probability
        action_log_prob = x_prob + y_prob + piece_prob + rotation_prob
        
        return [x.item(), y.item(), piece_type.item(), rotation.item()], action_log_prob.item(), value.item()
    
    def store_transition(self, state, action, action_log_prob, value, reward, done):
        self.memory.store_memory(state, action, action_log_prob, value, reward, done)
    
    def learn(self):
        for _ in range(self.n_epochs):
            # Calculate advantages and returns
            state_arr, action_arr, old_prob_arr, vals_arr, reward_arr, done_arr = self._process_memory()
            
            # Generate mini-batches
            batches = self.memory.generate_batches()
            
            # Train on each batch
            for batch in batches:
                # Select batch components
                grids = torch.FloatTensor(state_arr['grid'][batch]).to(self.device)
                orders = torch.FloatTensor(state_arr['order'][batch]).to(self.device)
                platform_idxs = torch.LongTensor(state_arr['platform_index'][batch]).to(self.device)
                
                actions = action_arr[batch]
                old_probs = old_prob_arr[batch]
                values = vals_arr[batch]
                
                # Calculate advantages and returns for the batch
                advantages = self._calculate_advantages(
                    reward_arr[batch], values, done_arr[batch]
                )
                returns = advantages + values
                advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)
                
                # Forward pass
                x_logits, y_logits, piece_logits, rotation_logits, critic_value = self.actor_critic(
                    grids, orders, platform_idxs
                )
                
                # Extract action components
                x = actions[:, 0]
                y = actions[:, 1]
                piece_type = actions[:, 2]
                rotation = actions[:, 3]
                
                # Create distributions
                x_dist = Categorical(F.softmax(x_logits, dim=1))
                y_dist = Categorical(F.softmax(y_logits, dim=1))
                piece_dist = Categorical(F.softmax(piece_logits, dim=1))
                rotation_dist = Categorical(F.softmax(rotation_logits, dim=1))
                
                # Calculate new log probabilities
                x_new_probs = x_dist.log_prob(torch.LongTensor(x).to(self.device))
                y_new_probs = y_dist.log_prob(torch.LongTensor(y).to(self.device))
                piece_new_probs = piece_dist.log_prob(torch.LongTensor(piece_type).to(self.device))
                rotation_new_probs = rotation_dist.log_prob(torch.LongTensor(rotation).to(self.device))
                
                # Combine log probabilities
                new_probs = x_new_probs + y_new_probs + piece_new_probs + rotation_new_probs
                
                # Calculate probability ratio
                prob_ratio = torch.exp(new_probs - torch.FloatTensor(old_probs).to(self.device))
                
                # Calculate surrogate losses
                weighted_probs = advantages.to(self.device) * prob_ratio
                weighted_clipped_probs = advantages.to(self.device) * torch.clamp(
                    prob_ratio, 1-self.policy_clip, 1+self.policy_clip
                )
                actor_loss = -torch.min(weighted_probs, weighted_clipped_probs).mean()
                
                # Add entropy bonus for exploration
                entropy = x_dist.entropy().mean() + y_dist.entropy().mean() + \
                        piece_dist.entropy().mean() + rotation_dist.entropy().mean()
                
                # Calculate critic loss
                returns = returns.float().to(self.device)  # Ensure returns is float32
                critic_loss = F.mse_loss(critic_value.squeeze(), returns)
                
                # Calculate total loss
                total_loss = actor_loss + 0.5 * critic_loss - self.entropy_coefficient * entropy
                
                # Update network
                self.optimizer.zero_grad()
                total_loss.backward()
                self.optimizer.step()
        
        # Clear memory after learning
        self.memory.clear_memory()

    def _calculate_advantages(self, rewards, values, dones):
        """Calculate advantages using Generalized Advantage Estimation (GAE)."""
        advantages = np.zeros_like(rewards)
        gae = 0
        
        for t in reversed(range(len(rewards))):
            if t == len(rewards) - 1:
                next_value = 0
            else:
                next_value = values[t + 1]
            
            delta = rewards[t] + self.gamma * next_value * (1 - dones[t]) - values[t]
            gae = delta + self.gamma * self.gae_lambda * (1 - dones[t]) * gae
            advantages[t] = gae
        
        return torch.tensor(advantages, dtype=torch.float32)  # Explicitly use float32
    
    def _process_memory(self):
        """Process memory data into arrays."""
        # States need special handling due to being dictionaries
        state_arr = {
            'grid': [],
            'order': [],
            'platform_index': []
        }
        
        for state in self.memory.states:
            state_arr['grid'].append(state['grid'])
            state_arr['order'].append(state['order'])
            state_arr['platform_index'].append(state['platform_index'])
        
        # Convert states to numpy arrays
        state_arr = {k: np.array(v) for k, v in state_arr.items()}
        
        # Convert other memory components to numpy arrays
        action_arr = np.array(self.memory.actions)
        old_prob_arr = np.array(self.memory.probs)
        vals_arr = np.array(self.memory.vals)
        reward_arr = np.array(self.memory.rewards)
        done_arr = np.array(self.memory.dones)
        
        return state_arr, action_arr, old_prob_arr, vals_arr, reward_arr, done_arr
    
    def _calculate_advantages(self, rewards, values, dones):
        """Calculate advantages using Generalized Advantage Estimation (GAE)."""
        advantages = np.zeros_like(rewards)
        gae = 0
        
        for t in reversed(range(len(rewards))):
            if t == len(rewards) - 1:
                next_value = 0
            else:
                next_value = values[t + 1]
            
            delta = rewards[t] + self.gamma * next_value * (1 - dones[t]) - values[t]
            gae = delta + self.gamma * self.gae_lambda * (1 - dones[t]) * gae
            advantages[t] = gae
        
        return torch.FloatTensor(advantages)
    
    def save_model(self, path):
        torch.save({
            'actor_critic': self.actor_critic.state_dict(),
            'optimizer': self.optimizer.state_dict()
        }, path)
    
    def load_model(self, path):
        checkpoint = torch.load(path, map_location=self.device)
        self.actor_critic.load_state_dict(checkpoint['actor_critic'])
        self.optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
import os
import time

def custom_order_example():
    """Example of using a custom order."""
    # Create environment
    env = WoodCuttingEnv()
    
    # Create and load agent
    agent = PPOAgent(
        state_shape={
            'grid': env.observation_space['grid'].shape,
            'order': env.observation_space['order'].shape,
            'platform_index': (1,)
        },
        action_space=env.action_space.nvec
    )
    
    try:
        agent.load_model(r'models/ppo_wood_cutting_final.pth')
        print("Loaded trained model")
    except:
        print("No trained model found, using untrained agent")
    
    multiple_orders = [
    [[40, 50, 2], [20, 25, 4], [4, 25, 10], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_001
    [[40, 50, 1], [20, 25, 6], [4, 25, 20], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_002
    [[40, 50, 3], [20, 25, 6], [4, 25, 25], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_003
    [[40, 50, 4], [20, 25, 8], [4, 25, 30], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_004
    [[40, 50, 5], [20, 25, 10], [4, 25, 35], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_005
    [[40, 50, 6], [20, 25, 12], [4, 25, 40], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_006
    [[40, 50, 7], [20, 25, 14], [4, 25, 45], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_007
    [[40, 50, 8], [20, 25, 16], [4, 25, 50], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_008
    [[40, 50, 4], [20, 25, 8], [4, 25, 30], [50, 60, 2], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_009
    [[40, 50, 5], [20, 25, 10], [4, 25, 35], [50, 60, 3], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_010
    [[40, 50, 6], [20, 25, 12], [4, 25, 40], [50, 60, 4], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_011
    [[40, 50, 7], [20, 25, 14], [4, 25, 45], [50, 60, 5], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_012
    [[40, 50, 3], [20, 25, 6], [4, 25, 20], [30, 70, 2], [60, 80, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_013
    [[40, 50, 4], [20, 25, 8], [4, 25, 25], [30, 70, 3], [60, 80, 2], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_014
    [[40, 50, 5], [20, 25, 10], [4, 25, 30], [30, 70, 4], [60, 80, 3], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_015
    [[40, 50, 6], [20, 25, 12], [4, 25, 35], [30, 70, 5], [60, 80, 4], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_016
    [[40, 50, 5], [20, 25, 10], [4, 25, 40], [50, 60, 2], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_017
    [[40, 50, 6], [20, 25, 12], [4, 25, 45], [30, 70, 3], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_018
    [[40, 50, 7], [20, 25, 14], [4, 25, 50], [60, 80, 2], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_019
    [[40, 50, 8], [20, 25, 16], [4, 25, 55], [50, 60, 3], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_020
    [[40, 50, 9], [20, 25, 18], [4, 25, 60], [30, 70, 4], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_021
    [[40, 50, 10], [20, 25, 20], [4, 25, 65], [60, 80, 3], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_022
    [[40, 50, 11], [20, 25, 22], [4, 25, 70], [50, 60, 4], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_023
    [[40, 50, 12], [20, 25, 24], [4, 25, 75], [30, 70, 5], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_024
    [[40, 50, 13], [20, 25, 26], [4, 25, 80], [60, 80, 4], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_025
    [[40, 50, 14], [20, 25, 28], [4, 25, 85], [50, 60, 5], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_026
    [[40, 50, 15], [20, 25, 30], [4, 25, 90], [30, 70, 6], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_027
    [[40, 50, 16], [20, 25, 32], [4, 25, 95], [60, 80, 5], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_028
    [[40, 50, 17], [20, 25, 34], [4, 25, 100], [50, 60, 6], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_029
    [[40, 50, 18], [20, 25, 36], [4, 25, 105], [30, 70, 7], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_030
    [[45, 55, 20], [22, 27, 40], [4, 26, 110], [32, 72, 8], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_031
    [[50, 60, 22], [24, 28, 45], [5, 27, 120], [35, 75, 9], [60, 80, 5], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_032
    [[55, 65, 24], [26, 30, 50], [5, 28, 130], [38, 78, 10], [65, 85, 6], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_033
    [[60, 70, 26], [28, 32, 55], [6, 30, 140], [40, 80, 11], [70, 90, 7], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_034
    [[65, 75, 28], [30, 34, 60], [6, 32, 150], [42, 82, 12], [75, 95, 8], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_035
    [[70, 80, 30], [32, 36, 65], [7, 34, 160], [45, 80, 13], [80, 80, 9], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_036
    [[75, 80, 32], [34, 38, 70], [7, 36, 170], [48, 20, 14], [80, 60, 10], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_037
    [[80, 80, 34], [36, 40, 75], [8, 38, 180], [50, 60, 15], [80, 40, 11], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_038
    [[50, 20, 36], [38, 42, 80], [8, 40, 190], [52, 80, 16], [30, 30, 12], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_039
    [[50, 10, 38], [40, 44, 85], [9, 42, 200], [55, 40, 17], [40, 50, 13], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_040
    [[50, 80, 40], [42, 46, 90], [9, 44, 210], [58, 80, 18], [50, 20, 14], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_041
    [[50, 60, 42], [44, 48, 95], [10, 46, 220], [60, 30, 19], [70, 30, 15], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_042
    [[50, 30, 44], [46, 50, 100], [10, 48, 230], [62, 40, 20], [30, 60, 16], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_043
    [[20, 80, 46], [48, 52, 105], [11, 50, 240], [65, 80, 21], [40, 30, 17], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_044
    [[20, 70, 48], [50, 54, 110], [11, 52, 250], [68, 43, 22], [70, 10, 18], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_045
    [[20, 40, 50], [52, 56, 115], [12, 54, 260], [70, 67, 23], [30, 50, 19], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_046
    [[20, 80, 52], [54, 58, 120], [12, 56, 270], [72, 38, 24], [30, 30, 20], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_047
    [[20, 80, 54], [56, 60, 125], [13, 58, 280], [75, 80, 25], [90, 30, 21], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_048
    [[50, 80, 56], [58, 62, 130], [13, 60, 290], [78, 80, 26], [34, 20, 22], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]],  # order_049
    [[50, 80, 58], [60, 64, 135], [14, 62, 300], [25, 80, 27], [20, 80, 23], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]   # order_050
]
    evaluating_history_file_path = fr'result/evaluating_history_data.npz'
    
    # Remove the old history file
    if os.path.exists(evaluating_history_file_path):
        os.remove(evaluating_history_file_path)

    waste_rate_history = []
    fitness_history = []
    platforms_used_history = []
    runtime_history = []

    for custom_order in multiple_orders:
        start_time = time.perf_counter()
        custom_order = np.array(custom_order)
        
        # Reset environment with custom order
        state = env.reset(order=custom_order)
        
        # Run episode with custom order
        done = False
        total_reward = 0
        steps = 0
        max_steps = 1000000

        # print(steps)
        # env.render()
        
        print("Starting custom order optimization...")
        
        while not done and steps < max_steps:
            # Choose action
            action, _, _  = agent.choose_action(state, training=False)
            
            # Take action
            next_state, reward, done, info = env.step(action)
            
            # Update state and total reward
            state = next_state
            total_reward += reward
            steps += 1
            # print("step",steps)
            # env.render()
            
            if done:
                end_time = time.perf_counter()
                runtime = end_time - start_time
                print(f"Order completed in {steps} steps")
                print(f"Total reward: {total_reward:.2f}")
                if 'waste_rate' in info:
                    waste_rate_history.append(info['waste_rate'])
                    fitness_history.append(info['fitness'])
                    platforms_used_history.append(info['platforms_used'])
                    runtime_history.append(runtime)
                    print(f"Waste Rate: {info['waste_rate']:.4f}")
                    print(f"Fitness: {info['fitness']:.4f}")
                    print(f"Platforms used: {info['platforms_used']}")
                    print(f"Filled Area: {info['filled_area']}")
                    print(f"Total Stock Area: {info['total_stock_area']}\n")
                np.savez(evaluating_history_file_path, 
                        waste_rate_history=waste_rate_history, 
                        fitness_history=fitness_history, 
                        platforms_used_history=platforms_used_history,
                        runtime_history=runtime_history)
                    
        # Render final state
        # print("step",steps)
        # env.render()
        
        if not done:
            print(f"Failed to complete order within {max_steps} steps")

In [ ]:
custom_order_example()

In [ ]:
data = np.load('result/evaluating_history_data.npz')

waste_rate_history = data['waste_rate_history']
fitness_history = data['fitness_history']
platforms_used_history = data['platforms_used_history']
runtime_history = data['runtime_history']


plt.figure(figsize=(15, 10))
    
plt.subplot(2, 2, 1)
plt.plot(waste_rate_history)
plt.title('Waste Rate Across Episodes')
plt.xlabel('Completed Episode')
plt.ylabel('Waste Rate')

plt.subplot(2, 2, 2)
plt.plot(fitness_history)
plt.title('Fitness Across Episodes')
plt.xlabel('Completed Episode')
plt.ylabel('Efficiency')

plt.subplot(2, 2, 3)
plt.plot(platforms_used_history)
plt.title('Platforms Used')
plt.xlabel('Completed Episode')
plt.ylabel('Number of Platforms')

plt.subplot(2, 2, 4)
plt.plot(runtime_history)
plt.title('Runtime Across Episodes')
plt.xlabel('Completed Episode')
plt.ylabel('Runtime')

plt.tight_layout()
plt.savefig(f"result/evaluating_on_50_customer_orders.png")
plt.show()